In [1]:
from num2words import num2words
from random import sample

In [36]:
def split_words(lst, day=True): 
    new_lst = []
    for word in lst:
        if day:
            new_lst.append("_DStart_ " + " ".join([char for char in word]) + " _DEnd_")
        else:
            new_lst.append("_MStart_ " + " ".join([char for char in word]) + " _MEnd_")
    return new_lst

In [37]:
month_ntow = split_words(["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"], False)
day_ntow = split_words([num2words(n) for n in range(1, 32)])
day_ntoo = split_words([num2words(n, ordinal=True) for n in range(1, 32)])

In [57]:
month_ntow[3]

'_MStart_ A p r i l _MEnd_'

In [60]:
from nltk import word_tokenize

In [72]:
def month_wtoz(month):
    length = len(month_ntow[month].split())
    return " ".join([str(2)] * length)

def day_wtoz(day):
    length = len(day_ntow[day].split())
    return " ".join([str(0)] * length)

def day_otoz(day):
    length = len(day_ntoo[day].split())
    return " ".join([str(1)] * length)

In [79]:
def date_to_sent(day, month, year, format, start_format):
    day -= 1
    month -= 1

    starts = [("The date is ", "-1 -1 -1 "), ("It is ", "-1 -1 "), ("Today is ", "-1 -1 ")][start_format]
    
    y = starts[0]
    z = starts[1]
    
    if format == 0:
        y += day_ntow[day] + " " + month_ntow[month] + " " + str(year)
        z += day_wtoz(day) + " " + month_wtoz(month) + " " + "3"
    elif format == 1:
        y += month_ntow[month] + " " + day_ntow[day] + " , " + str(year)
        z += month_wtoz(month) + " " + day_wtoz(day) + " -1 " + "3"
    elif format == 2:
        y += "the " + day_ntoo[day] + " of " + month_ntow[month] + " , " + str(year)
        z += "-1 " + day_otoz(day) + " -1 " + month_wtoz(month) + " -1 " + "3"
    elif format == 3:
        y += month_ntow[month] + " the " + day_ntoo[day] + " , " + str(year)
        z += month_wtoz(month) + " -1 " + day_otoz(day) + " -1 " + "3"
    elif format == 4:
        y += month_ntow[month] + " " + day_ntoo[day] + " of the year " + str(year)
        z += month_wtoz(month) + " " + day_otoz(day) + " -1 -1 -1 " + "3"
    elif format == 5:
        y += "the " + day_ntoo[day] + " of " + month_ntow[month] + " in the year " + str(year)
        z += "-1 " + day_otoz(day) + " -1 " + month_wtoz(month) + " -1 -1 -1 " + "3"
        
    y += " ."
    z += " -1"
    assert len(y.split(" ")) == len(z.split(" "))

    return y, z

In [80]:
days = list(range(1,32))
months = list(range(1,13))
years = list(range(2000,2021))
formats = list(range(6))
start_formats = list(range(3))

In [83]:
def get_data(sz):
    data = []

    for _ in range(sz):
        day = sample(days, 1)[0]
        month = sample(months, 1)[0]
        year = sample(years, 1)[0]
        format = sample(formats, 1)[0]
        start_format = sample(start_formats, 1)[0]

        y,z = date_to_sent(day, month, year, format, start_format)

        data.append(((day, month, year), y, z, format, start_format))

    return data

In [84]:
train = get_data(2000)
dev = get_data(1000)
test = get_data(1000)

In [94]:
import pickle
pickle.dump((train, dev, test), open("data_raw.pkl", "wb"))

In [86]:
def print_datapoint(data):
    (day, month, year), y, z, _, _ = data
    if day < 10:
        day = "0 " + str(day)
    else:
        day = " ".join([el for el in str(day)])
    txt = "\"day[" + str(day) + "], month[" + str(month) + "], year[" + str(year) + "]\"," + "\"" + y + "\""
    return txt

def print_datapointz(data):
    (day, month, year), y, z, _, _ = data
    if day < 10:
        day = "0 " + str(day)
    else:
        day = " ".join([el for el in str(day)])
    txt = "\"day[" + str(day) + "], month[" + str(month) + "], year[" + str(year) + "]\"," + "\"" + y + "\""
    txt += "," + "\"" + z + "\""
    return txt

def print_wo_ref(data):
    (day, month, year), y, z, _, _ = data
    if day < 10:
        day = "0 " + str(day)
    else:
        day = " ".join([el for el in str(day)])
    txt = "\"day[" + str(day) + "], month[" + str(month) + "], year[" + str(year) + "]\""
    return txt

In [89]:
i = 0
for datapoint in train:
    print(print_datapointz(datapoint))
    i += 1
    if i == 10:
        break

"day[0 8], month[11], year[2000]","It is the _DStart_ e i g h t h _DEnd_ of _MStart_ N o v e m b e r _MEnd_ in the year 2000 .","-1 -1 -1 1 1 1 1 1 1 1 1 -1 2 2 2 2 2 2 2 2 2 2 -1 -1 -1 3 -1"
"day[0 1], month[1], year[2001]","The date is the _DStart_ f i r s t _DEnd_ of _MStart_ J a n u a r y _MEnd_ in the year 2001 .","-1 -1 -1 -1 1 1 1 1 1 1 1 -1 2 2 2 2 2 2 2 2 2 -1 -1 -1 3 -1"
"day[2 9], month[6], year[2000]","It is the _DStart_ t w e n t y - n i n t h _DEnd_ of _MStart_ J u n e _MEnd_ , 2000 .","-1 -1 -1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 -1 2 2 2 2 2 2 -1 3 -1"
"day[0 7], month[9], year[2009]","It is the _DStart_ s e v e n t h _DEnd_ of _MStart_ S e p t e m b e r _MEnd_ , 2009 .","-1 -1 -1 1 1 1 1 1 1 1 1 1 -1 2 2 2 2 2 2 2 2 2 2 2 -1 3 -1"
"day[1 3], month[7], year[2001]","Today is _MStart_ J u l y _MEnd_ _DStart_ t h i r t e e n _DEnd_ , 2001 .","-1 -1 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 -1 3 -1"
"day[0 2], month[9], year[2015]","The date is _MStart_ S e p t e m b e r _MEnd_ _DStart_ s e 

In [95]:
with open("trainset.csv", "w") as ofile:
    print("mr,ref", file=ofile)
    for datapoint in train:
        print(print_datapointz(datapoint), file=ofile)

In [96]:
with open("devset.csv", "w") as ofile:
    print("mr,ref", file=ofile)
    for datapoint in dev:
        print(print_datapointz(datapoint), file=ofile)

In [97]:
with open("testset.csv", "w") as ofile:
    print("MR", file=ofile)
    for datapoint in train:
        print(print_wo_ref(datapoint), file=ofile)

In [98]:
with open("testset_w_refs.csv", "w") as ofile:
    print("mr,ref", file=ofile)
    for datapoint in train:
        print(print_datapointz(datapoint), file=ofile)